In [ ]:
import numpy as np

KAPPA_MAG = np.arctanh(np.sqrt(0.95)) / 2000 * 2 / np.pi


def get_L(delta_k1, delta_k2):
    """線形演算子Lに対応する対角行列の要素を返す"""
    return 1j * np.array([0.0, delta_k1, delta_k1 + delta_k2])


def N(B, kappa):
    """非線形項N(B)を計算する"""
    B1, B2, B3 = B
    return 1j * kappa * np.array([
        np.conj(B1) * B2 + np.conj(B2) * B3,
        B1**2 + 2 * np.conj(B1) * B3,
        3 * B1 * B2
    ])


def A_from_B(B, z, delta_k1, delta_k2):
    """正準変数Bから物理的な振幅Aに変換する"""
    phase_factors = np.exp(-1j *
                           np.array([0, delta_k1 * z, (delta_k1 + delta_k2) * z]))
    return B * phase_factors


def generate_periodic_domains(z_start, z_end, period, kappa_mag):
    """周期的な符号反転ドメイン構造を生成する"""
    domains = []
    if period < 1e-12:
        return domains
    half_period = period / 2.0
    current_z = z_start
    sign_flipper = 1.0
    while current_z + half_period <= z_end:
        domains.append((half_period, sign_flipper * kappa_mag))
        current_z += half_period
        sign_flipper *= -1.0
    return domains


def calculate_coeffs(L, h):
    """ETDRK4法の係数を計算する"""
    z = h * L
    exp_z = np.exp(z)
    exp_z_half = np.exp(z / 2.0)
    z2, z3 = z**2, z**3
    mask = np.abs(z) < 1e-8

    # テイラー展開によるゼロ割の回避
    Q = h * np.divide(exp_z_half - 1.0, z, where=~mask)
    f1 = h * np.divide(-4 - z + exp_z * (4 - 3*z + z2), z3, where=~mask)
    f2 = h * np.divide(2 * (2 + z + exp_z * (-2 + z)), z3, where=~mask)
    f3 = h * np.divide(-4 - 3*z - z2 + exp_z * (4 - z), z3, where=~mask)

    Q[mask] = h / 2.0
    f1[mask] = h / 6.0
    f2[mask] = h / 3.0
    f3[mask] = h / 6.0

    return exp_z, exp_z_half, Q, f1, f2, f3


def predictor_etdrk4(B_in, h, kappa_val, L, subdivisions):
    """予測子: ETDRK4法を用いて1ドメイン内の時間発展を計算"""
    B = B_in.copy()
    h_step = h / subdivisions
    if h_step < 1e-12:
        return B
    E, E2, Q, f1, f2, f3 = calculate_coeffs(L, h_step)
    for _ in range(subdivisions):
        NB = N(B, kappa_val)
        a = E2 * B + Q * NB
        Na = N(a, kappa_val)
        b = E2 * B + Q * Na
        Nb = N(b, kappa_val)
        c = E2 * a + Q * (2 * Nb - NB)
        Nc = N(c, kappa_val)
        B = E * B + (f1 * NB + f2 * (Na + Nb) + f3 * Nc)
    return B


def hamiltonian_K(B, kappa, delta_k1, delta_k2):
    """ハミルトニアンKを計算する"""
    B1, B2, B3 = B
    k_shg = kappa * np.real(B1**2 * np.conj(B2))
    k_sfg = 2 * kappa * np.real(B1 * B2 * np.conj(B3))
    k_shift = (delta_k1 / 2.0) * np.abs(B2)**2 + \
              ((delta_k1 + delta_k2) / 3.0) * np.abs(B3)**2
    return k_shg + k_sfg + k_shift


def gradient_K(B, kappa, delta_k1, delta_k2):
    """ハミルトニアンKのB*に対する勾配を計算する"""
    B1, B2, B3 = B
    grad_B1_conj = kappa * (np.conj(B1) * B2 + np.conj(B2) * B3)
    grad_B2_conj = kappa * (0.5 * B1**2 + np.conj(B1)
                            * B3) + (delta_k1 / 2.0) * B2
    grad_B3_conj = kappa * (B1 * B2) + ((delta_k1 + delta_k2) / 3.0) * B3
    return np.array([grad_B1_conj, grad_B2_conj, grad_B3_conj])


def corrector_IK_projection(B_pred, B_in, kappa, delta_k1, delta_k2, I_initial):
    """修正子: 全光強度IとハミルトニアンKを保存するように射影"""
    # ステップ開始時の目標値
    K_target = hamiltonian_K(B_in, kappa, delta_k1, delta_k2)
    I_target = I_initial

    # 予測値
    K_pred = hamiltonian_K(B_pred, kappa, delta_k1, delta_k2)
    I_pred = np.sum(np.abs(B_pred)**2)

    # 誤差
    e_I = I_pred - I_target
    e_K = K_pred - K_target

    # 誤差が十分に小さければ補正は不要
    if np.abs(e_I) < 1e-15 and np.abs(e_K) < 1e-15:
        return B_pred

    # 予測点における勾配
    grad_I = B_pred
    grad_K_val = gradient_K(B_pred, kappa, delta_k1, delta_k2)

    # ラグランジュ未定乗数を求めるための2x2行列Mを構築
    M = np.zeros((2, 2), dtype=float)
    M[0, 0] = np.real(np.vdot(grad_I, grad_I))
    M[0, 1] = np.real(np.vdot(grad_I, grad_K_val))
    M[1, 0] = M[0, 1]
    M[1, 1] = np.real(np.vdot(grad_K_val, grad_K_val))

    # 誤差ベクトル
    e_vec = 0.5 * np.array([e_I, e_K])

    # Mλ = e を解き、未定乗数λを求める
    try:
        # 堅牢性のために最小二乗法ソルバーを使用
        lambdas, _, _, _ = np.linalg.lstsq(M, e_vec, rcond=None)
    except np.linalg.LinAlgError:
        # 行列が特異であるなど、求解に失敗した場合は強度補正のみにフォールバック
        scaling_factor = np.sqrt(I_target / I_pred) if I_pred > 1e-16 else 1.0
        return scaling_factor * B_pred

    lambda_I, lambda_K = lambdas

    # 状態ベクトルを補正
    B_corr = B_pred - lambda_I * grad_I - lambda_K * grad_K_val

    return B_corr


def simulate_superlattice(B0, domains, L, delta_k1, delta_k2, subdivisions_per_domain, use_corrector=False):
    """超格子構造全体のシミュレーションを実行する"""
    z = 0.0
    B = B0.astype(complex)
    I_initial = np.sum(np.abs(B)**2)

    for h, kappa_val in domains:
        if h < 1e-12:
            continue

        # ドメインステップ開始時の状態を保持
        B_in = B.copy()

        B_pred = predictor_etdrk4(B, h, kappa_val, L, subdivisions_per_domain)

        if use_corrector:
            B = corrector_IK_projection(
                B_pred, B_in, kappa_val, delta_k1, delta_k2, I_initial)
        else:
            B = B_pred

        z += h

    return z, B


def get_scenarios(kappa_mag):
    """シミュレーションシナリオを定義する"""
    DELTA_K1_SHG = 2 * np.pi / 7.2
    DELTA_K2_SFG = 3.2071
    period_shg = 2 * np.pi / DELTA_K1_SHG
    # 高次準位相整合でもシミュレーションに成功するかどうかが大事
    period_higher_order_shg = period_shg * 3
    period_sfg = 2 * np.pi / DELTA_K2_SFG
    Z_MAX_1, Z_MAX_2, Z_MAX_3, Z_SPLIT = 6000.0, 2400.0, 4400.0, 2000.0

    return [
        {
            "name": f"Case 1: QPM for Higher Order SHG (Λ={period_higher_order_shg:.2f} μm)",
            "domains": generate_periodic_domains(0.0, Z_MAX_1, period_higher_order_shg, kappa_mag),
            "A0": np.array([1.0, 0.0, 0.0]),
            "delta_k1": DELTA_K1_SHG, "delta_k2": DELTA_K2_SFG
        },
        {
            "name": f"Case 2: QPM for SFG (Λ≈{period_sfg:.2f} μm)",
            "domains": generate_periodic_domains(0.0, Z_MAX_2, period_sfg, kappa_mag),
            "A0": np.array([np.sqrt(0.5), np.sqrt(0.5), 0.0]),
            "delta_k1": DELTA_K1_SHG, "delta_k2": DELTA_K2_SFG
        },
        {
            "name": "Case 3: Cascaded QPM for THG",
            "domains": (
                generate_periodic_domains(0.0, Z_SPLIT, period_shg, kappa_mag) +
                generate_periodic_domains(
                    Z_SPLIT, Z_MAX_3, period_sfg, kappa_mag)
            ),
            "A0": np.array([1.0, 0.0, 0.0]),
            "delta_k1": DELTA_K1_SHG, "delta_k2": DELTA_K2_SFG
        }
    ]


if __name__ == "__main__":
    scenarios = get_scenarios(KAPPA_MAG)
    SUBDIVISIONS_PER_DOMAIN = 1  # **ここは1にしてください**

    for config in scenarios:
        print(f"\n--- {config['name']} ---")
        L = get_L(config["delta_k1"], config["delta_k2"])
        initial_I = np.sum(np.abs(config["A0"])**2)

        final_z_corr, final_B_corr = simulate_superlattice(
            config["A0"], config["domains"], L, config["delta_k1"], config["delta_k2"],
            SUBDIVISIONS_PER_DOMAIN, use_corrector=True
        )
        final_A_corr = A_from_B(
            final_B_corr, final_z_corr, config["delta_k1"], config["delta_k2"])
        final_I_corr = np.abs(final_A_corr)**2

        print(f"Final z: {final_z_corr:.2f} μm")
        print(
            f"Final Intensities (I1, I2, I3): {final_I_corr[0]:.8f}, {final_I_corr[1]:.8f}, {final_I_corr[2]:.8f}")
        print(
            f"Total Intensity: {np.sum(final_I_corr):.8f} (Error: {np.sum(final_I_corr) - initial_I:.2e})")